# Model Inference

In [ ]:
import pyhf
import json

In [ ]:
with open("data/2-bin_1-channel.json") as serialized:
  spec = json.load(serialized)

workspace = pyhf.Workspace(spec)
model = workspace.model(poi_name="mu")

In [ ]:
pars = model.config.suggested_init()
data = workspace.data(model)

Creating the log-likelihood conditioned on the data
$$
L(\vec{\theta}|\vec{x}) = k \cdot p(\vec{x} | \vec{\theta})
$$

In [ ]:
model.logpdf(pars, data)

moar inference

In [ ]:
bestfit_pars, twice_nll = pyhf.infer.mle.fit(data, model, return_fitted_val=True)
print(bestfit_pars)

objective function is twice the negative log-likelihood

In [ ]:
-2 * model.logpdf(bestfit_pars, data) == twice_nll